In [1]:
# General purpose
import numpy as np
import time
from pathlib import Path
from glob import glob

# Segmentation
from cellpose_omni import models, core, io, transforms
from omnipose.utils import normalize99

2025-01-08 20:31:35,394 [INFO] No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'


In [2]:
# define parameters
params = {'rescale': None, # upscale or downscale your images, None = no rescaling 

          'mask_threshold': 0, # erode or dilate masks with higher or lower values 

          'flow_threshold': 0.4, # default is .4, but only needed if there are spurious masks to clean up; slows down output

          'transparency': True, # transparency in flow output

          'omni': True, # we can turn off Omnipose mask reconstruction, not advised 

          'cluster': True, # use DBSCAN clustering

          'resample': True, # whether or not to run dynamics on rescaled grid or original grid 

          # 'verbose': False, # turn on if you want to see more output 

          'tile': False, # average the outputs from flipped (augmented) images; slower, usually not needed 

          'niter': None, # None lets Omnipose calculate # of Euler iterations (usually <20) but you can tune it for over/under segmentation 

          'augment': False, # Can optionally rotate the image and average outputs, usually not needed 

          'affinity_seg': False, # new feature, stay tuned...

         }

model_path = 'Desktop/models/FMDeconvolved_2D_Omnipose'
model = models.CellposeModel(pretrained_model=model_path, nclasses=3, nchan=1, gpu=False, diam_mean=0)

2025-01-08 20:31:37,322 [INFO] >>>> using CPU
2025-01-08 20:31:37,323 [INFO] WARNING: MKL version on torch not working/installed - CPU version will be slightly slower.
2025-01-08 20:31:37,324 [INFO] see https://pytorch.org/docs/stable/backends.html?highlight=mkl


In [4]:

im_path = 'Desktop/Test_images_deconvolved/*.tif'

files = sorted(glob(im_path))

imgs = [normalize99(io.imread(f)) for f in files]

print(f'\nnumber of images: {len(imgs)}')


number of images: 16


In [ ]:
tic = time.time() 

masks, flows, styles = model.eval([img for img in imgs],**params)

net_time = time.time() - tic

print('total segmentation time: {}s'.format(net_time))

  0%|          | 0/16 [00:00<?, ?it/s]

In [7]:
# Save masks and outlines

io.save_masks(imgs, masks, flows, files, 

              png=False,

              tif=True, # whether to use PNG or TIF format

              suffix='', # suffix to add to files if needed 

              save_flows=False, # saves both RGB depiction as *_flows.png and the raw components as *_dP.tif

              save_outlines=True, # save outline images 

              dir_above=1, # save output in the image directory or in the directory above (at the level of the image directory)

              in_folders=True, # save output in folders (recommended)

              save_txt=False, # txt file for outlines in imageJ

              save_ncolor=False) # save ncolor version of masks for visualization and editing 
